## My Personal Data Cleaning Thought Process

#### This notebook documents my step-by-step approach to cleaning the Life Expectancy dataset before building linear regression models (both single and multivariate). I'll include my thought process, decisions, and reasoning behind each cleaning step.



## Step 1: Initial Data Exploration
* Goal: Understand the dataset's structure, missing values, and potential issues.



In [1]:
# Import necessary libraries
import pandas as pd

# Load the dataset
df = pd.read_csv('Life_Expectancy/Life Expectancy Data.csv')


# First, let's see the shape and columns
print("Dataset shape:", df.shape)  # How many rows and columns?
print("\nFirst 5 rows:")
df.head()  # Peek at the data structure

Dataset shape: (2938, 22)

First 5 rows:


,Country,Year,Status,Life expectancy,Adult Mortality,infant deaths,Alcohol,percentage expenditure,Hepatitis B,Measles,...,Polio,Total expenditure,Diphtheria,HIV/AIDS,GDP,Population,thinness 1-19 years,thinness 5-9 years,Income composition of resources,Schooling
0,Afghanistan,2015,Developing,65.0,263.0,62,0.01,71.279624,65.0,1154,...,6.0,8.16,65.0,0.1,584.259210,33736494.0,17.2,17.3,0.479,10.1
1,Afghanistan,2014,Developing,59.9,271.0,64,0.01,73.523582,62.0,492,...,58.0,8.18,62.0,0.1,612.696514,327582.0,17.5,17.5,0.476,10.0
2,Afghanistan,2013,Developing,59.9,268.0,66,0.01,73.219243,64.0,430,...,62.0,8.13,64.0,0.1,631.744976,31731688.0,17.7,17.7,0.470,9.9
3,Afghanistan,2012,Developing,59.5,272.0,69,0.01,78.184215,67.0,2787,...,67.0,8.52,67.0,0.1,669.959000,3696958.0,17.9,18.0,0.463,9.8
4,Afghanistan,2011,Developing,59.2,275.0,71,0.01,7.097109,68.0,3013,...,68.0,7.87,68.0,0.1,63.537231,2978599.0,18.2,18.2,0.454,9.5


## Observations & Thoughts:
* The dataset has 2938 rows and 22 columns (that's quite a bit)

* Some columns like Life expectancy, Adult Mortality, and GDP seem numerical.

* Country and Status are categorical.

### Next: Check for missing values and duplicates.

In [2]:
# Check missing values
print("\nMissing values per column:")
print(df.isnull().sum())

# Check for duplicates
print("\nNumber of duplicate rows:", df.duplicated().sum())


Missing values per column:
Country                              0
Year                                 0
Status                               0
Life expectancy                     10
Adult Mortality                     10
infant deaths                        0
Alcohol                            194
percentage expenditure               0
Hepatitis B                        553
Measles                              0
 BMI                                34
under-five deaths                    0
Polio                               19
Total expenditure                  226
Diphtheria                          19
 HIV/AIDS                            0
GDP                                448
Population                         652
 thinness  1-19 years               34
 thinness 5-9 years                 34
Income composition of resources    167
Schooling                          163
dtype: int64

Number of duplicate rows: 0


## Thoughts on Missing Data:
Some columns (Hepatitis B, GDP, Population) have missing values.

Decision:

* If a column has few missing values, I might fill them with median/mean.

* If a column has too many missing values, I might drop it.

* First, I should analyze how missingness affects Life expectancy.



### Handle Missing Values
Goal: Decide how to deal with missing data without distorting the analysis.

## Before handling the missing values, I will fix some improper space use in the column names

In [3]:
# I will handle some space problems with the column names that I have observed
df.columns = df.columns.str.replace('  ', ' ') 
df.columns = df.columns.str.strip()# I had a hard time getting this to work
#df.columns = [col.strip() for col in df.columns]

### Now, I will check and compare percentages of the missing errors in each column and decide to fill or drop it

In [4]:
missing_percent = df.isnull().mean() * 100
print('Missing Values (%): ')
print(missing_percent[missing_percent > 0].sort_values(ascending=False)) # Display missing values in percentages (From highest to lowest)

Missing Values (%): 
Population                         22.191967
Hepatitis B                        18.822328
GDP                                15.248468
Total expenditure                   7.692308
Alcohol                             6.603131
Income composition of resources     5.684139
Schooling                           5.547992
BMI                                 1.157250
thinness 1-19 years                 1.157250
thinness 5-9 years                  1.157250
Polio                               0.646698
Diphtheria                          0.646698
Life expectancy                     0.340368
Adult Mortality                     0.340368
dtype: float64


### Thoughts on result (missing values percentages):
* Population has approximately 22% null values which is a high value and filling it may cause a change in the columns distribution (I'll most likely drop it rather than fill it with mean or mode)
* I'm also thinking of setting my highest aceptable percent for filling to be around 15%(I actually feel like there will be a sort of relationship between GDP and Life Expectancy That's why i will fill the missing values with mean or mode.)
* I will also drop the Hepatitis B column since the percentage is more close to 19 

In [5]:
# Drop 'Population' and 'Hepatitis' B columns.
df.drop(columns=['Population','Hepatitis B'], inplace=True)
df.head()
df.shape

(2938, 20)

* Now to fill the some other columns

In [6]:
# fill other null values with mean
columns_to_fill = df.isnull().mean()
df.fillna(columns_to_fill, inplace=True)
df.isnull().sum()

Country                            0
Year                               0
Status                             0
Life expectancy                    0
Adult Mortality                    0
infant deaths                      0
Alcohol                            0
percentage expenditure             0
Measles                            0
BMI                                0
under-five deaths                  0
Polio                              0
Total expenditure                  0
Diphtheria                         0
HIV/AIDS                           0
GDP                                0
thinness 1-19 years                0
thinness 5-9 years                 0
Income composition of resources    0
Schooling                          0
dtype: int64

# Step 2: Remove duplicates:
## This should have been done earlier.

In [7]:
# Drop duplicates
df.drop_duplicates(inplace=True)
df.shape
df.dtypes

Country                             object
Year                                 int64
Status                              object
Life expectancy                    float64
Adult Mortality                    float64
infant deaths                        int64
Alcohol                            float64
percentage expenditure             float64
Measles                              int64
BMI                                float64
under-five deaths                    int64
Polio                              float64
Total expenditure                  float64
Diphtheria                         float64
HIV/AIDS                           float64
GDP                                float64
thinness 1-19 years                float64
thinness 5-9 years                 float64
Income composition of resources    float64
Schooling                          float64
dtype: object

# Step 3: Handle Outliers 
### I plan to cap the outliers using the 99th quantile as the upper limit and the 1st quantile as the lower limit
* since i want to check the outliers for all the columns, I will use a for loop
* in the Exploratory analysis notebook, I will visualize and tune it more if needed.

In [8]:
num_cols = df.select_dtypes(include=['int64', 'float64']).columns
for col in num_cols:
    upper_limit = df[col].quantile(0.99)
    lower_limit = df[col].quantile(0.01)
    df[col] = df[col].clip(lower= lower_limit, upper= upper_limit)

# Step 4: Encoding
#### I will encode the countries and status of the country (it may be of significance)

In [12]:
cat_cols = df.select_dtypes(include= ['object']).astype(str)
#for col in cat_cols:
 #   try:
  #      df = pd.get_dummies(df, columns=cat_cols[col])
   # except ValueError as e:
    #    print(f'Error occured while one-hot encoding column {col}: {e}')
cols_to_encode = [col for col in cat_cols if col in df.columns]
df_encoded = pd.get_dummies(df, cols_to_encode)
df_encoded.head()
df_encoded.to_csv('encoded_data.csv')

In [13]:
df.head()
df.to_csv('cleaned_data.csv')